In [1]:
from __future__ import division, generators, absolute_import, print_function

import numpy as np
import scipy.stats
import pandas as pd

data_dir = '../data/'

# Read Data

In [2]:
def read_data(data_dir, file_name, skiprows, triple_col):
    """
    This function reads data from xlsx file, and remove triples with 
    at least one missing read counts in the triple
    
    Parameters
    ----------
        data_dir: string
            data directory
        file_name: string
            xlsx file name
        skiprows: int
            rows to skip at the beginning
        triple_col: list of string
            a list of length 3, the column name for the triple in the xlsx file
    
    Returns
    -------
        df: pandas.core.frame.DataFrame
            pandas data frame with triple column named col1, col2 and col3
        filtered_df: pandas.core.frame.DataFrame
            pandas data frame with triple column named col1, col2 and col3,
            each triple is sorted by increasing order,
            triples with gap less than or equal to 1 are filtered out
    """
    df = pd.read_excel(data_dir + file_name, skiprows = skiprows)
    df[triple_col] = df[triple_col].apply(
        lambda x: pd.to_numeric(x, errors='coerce'))
    nan_index = df[triple_col].apply(np.isnan).any(1).nonzero()[0]
    col_name = ['col1', 'col2', 'col3']
    df = df.rename(columns = {triple_col[0]: col_name[0],
                              triple_col[1]: col_name[1],
                              triple_col[2]: col_name[2]})
    df = df.drop(df.index[nan_index])
    df = df.reset_index(drop=True)
    
    # sort each triple by increasing order
    filtered_df = df.copy(deep=True)
    filtered_df[col_name] = filtered_df[col_name].apply(np.sort, axis = 1)
    
    # filter out the triples with gap less than or equal to 1
    complete_index = filtered_df[col_name].apply(
        lambda x: x.col3 - x.col1 <=1, axis=1).nonzero()[0]
    filtered_df = filtered_df.drop(filtered_df.index[complete_index])
    filtered_df = filtered_df.reset_index(drop=True)
    
    return df, filtered_df

In [3]:
# colony data: RTS
rts_colony, filtered_rts_colony = read_data(
    data_dir, 'Bishayee Colony Counts 10.27.97-3.8.01.xlsx', 
    skiprows=2, triple_col=['col1', 'col2', 'col3'])

# colony data: other invectigators in the lab
inv_colony, filtered_inv_colony = read_data(
    data_dir, 'Other Investigators in Lab.Colony Counts.4.23.92-11.27.02.xlsx', 
    skiprows=1, triple_col=['col1', 'col2', 'col3'])

# colony data: outside lab
lab_colony, filtered_lab_colony = read_data(
    data_dir, 'Outside Lab 3.Colony Counts.2.4.10-5.21.12.xlsx', 
    skiprows=1, triple_col=['c1', 'c2', 'c3'])

In [4]:
# coulter data: RTS
rts_coulter, filtered_rts_coulter = read_data(
    data_dir, 'Bishayee Coulter Counts.10.20.97-7.16.01.xlsx', 
    skiprows=1, triple_col=['Count 1', 'Count 2', 'Count 3'])

# coulter data: other invectigators in the lab
inv_coulter, filtered_inv_coulter = read_data(
    data_dir, 'Other Investigators in Lab.Coulter Counts.4.15.92-5.21.05.xlsx', 
    skiprows=1, triple_col=['Coul 1', 'Coul 2', 'Coul 3'])

# coulter data: outside lab1
lab1_coulter, filtered_lab1_coulter = read_data(
    data_dir, 'Outside Lab 1.Coulter Counts.6.7.91-4.9.99.xlsx', 
    skiprows=0, triple_col=['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3'])

# coulter data: outside lab2
lab2_coulter, filtered_lab2_coulter = read_data(
    data_dir, 'Outside Lab 2.Coulter Counts.6.6.08-7.7.08.xlsx', 
    skiprows=1, triple_col=['Count 1', 'Count 2', 'Count 3'])

In [5]:
print("Number of triples in RTS colony data: ", rts_colony.shape[0], 
      "(after filtering:", filtered_rts_colony.shape[0], ")")
print("Number of triples in other invectigators colony data: ", inv_colony.shape[0],
      "(after filtering:", filtered_inv_colony.shape[0], ")")
print("Number of triples in outside lab colony data: ", lab_colony.shape[0],
      "(after filtering:", filtered_lab_colony.shape[0], ")")
print("Number of triples in RTS coulter data: ", rts_coulter.shape[0],
      "(after filtering:", filtered_rts_coulter.shape[0], ")")
print("Number of triples in other invectigators coulter data: ", inv_coulter.shape[0],
      "(after filtering:", filtered_inv_coulter.shape[0], ")")
print("Number of triples in outside lab 1 coulter data: ", lab1_coulter.shape[0],
      "(after filtering:", filtered_lab1_coulter.shape[0], ")")
print("Number of triples in outside lab 2 coulter data: ", lab2_coulter.shape[0],
      "(after filtering:", filtered_lab2_coulter.shape[0], ")")

Number of triples in RTS colony data:  1361 (after filtering: 1343 )
Number of triples in other invectigators colony data:  597 (after filtering: 578 )
Number of triples in outside lab colony data:  50 (after filtering: 49 )
Number of triples in RTS coulter data:  1727 (after filtering: 1726 )
Number of triples in other invectigators coulter data:  929 (after filtering: 929 )
Number of triples in outside lab 1 coulter data:  97 (after filtering: 97 )
Number of triples in outside lab 2 coulter data:  120 (after filtering: 120 )


# Read Mid-probability table

In [6]:
# read the mid probability table from output directory
# mid probability table: the mid-containing probability for different 
# values of lambdas
output_dir = "../outputs/"
midProb = np.loadtxt(output_dir + "MidProb.txt", delimiter="\t")
midProb.shape

(2129, 2)

# Hypothesis III

In [7]:
def hypothesis_3(filtered_df, midProb, print_res=False):
    """
    This function implements the hypothesis test 3 for mean-containing 
    ratio in the paper.
    
    Parameters
    ----------
        filtered_df: pandas.core.frame.DataFrame
            pandas data frame with triple column named col1, col2 and col3,
            each triple is sorted by increasing order,
            triples with gap less than or equal to 1 are filtered out
        midProb: numpy.ndarray
            mid probability table, the mid-containing probability for 
            different values of lambdas
        print_res: default False
            False: print all the results
            True: return number of mean containing triples, mean and sd
                of poission binomial, Z statistic and p-value without printing
    """
    # sort each triple by increasing order
    col_name = ['col1', 'col2', 'col3']
    filtered_df[col_name] = filtered_df[col_name].apply(np.sort, axis = 1)
    
    # whether the mid number is the mean of largest and smallest
    isMid = filtered_df.apply(
        lambda x: x.col1 + x.col3 in [2*x.col2, 2*x.col2 + 1, 2*x.col2 - 1], axis=1)
    
    # rounded mean triple as the estimate of lambda
    lamEst = np.mean(filtered_df[col_name], axis=1)
    # probabiliy that the triple been randomly drawn it would contain its own mean
    lams = np.array([i[0] for i in midProb])
    probs = [midProb[np.argmin(abs(lams-i))][1] for i in lamEst]
    
    # expectation and variance of poisson binomial
    mu = np.sum(probs)
    sigma2 = np.sum(p*(1-p) for p in probs)
    
    # compute statistic and p-value for hypothesis 3
    stat3 = (sum(isMid) - 0.5 - mu) / np.sqrt(sigma2)
    p_value3 = 1 - scipy.stats.norm.cdf(stat3)
    
    if print_res:
        print("Number of mean containing triples: ", sum(isMid))
        print("Mean of poission binomial: ", mu)
        print("SD of poission binomial: ", np.sqrt(sigma2))
        print("Z statistic: ", stat3)
        print("p-value: ", p_value3)
    else:
        return len(filtered_df), sum(isMid), mu, np.sqrt(sigma2), stat3, p_value3

## Reproduce results

In [8]:
hypothesis_3_res = pd.DataFrame(columns=('No. complete', 'No. mean containing', 
                                         'No.expected', "sd", "Z", "p-value"))

In [9]:
for dfs in ["filtered_rts_colony", "filtered_inv_colony", "filtered_lab_colony",
            "filtered_rts_coulter", "filtered_inv_coulter", 
            "filtered_lab1_coulter", "filtered_lab2_coulter"]:
    hypothesis_3_res.loc[dfs] = hypothesis_3(eval(dfs), midProb)

In [10]:
hypothesis_3_res

,No. complete,No. mean containing,No.expected,sd,Z,p-value
filtered_rts_colony,1343.0,690.0,214.923602,13.281792,35.731353,0.000000e+00
filtered_inv_colony,578.0,109.0,103.404567,9.061666,0.562306,2.869536e-01
filtered_lab_colony,49.0,3.0,7.788269,2.554401,-2.070258,9.807859e-01
filtered_rts_coulter,1726.0,176.0,98.367748,9.612620,8.024061,5.551115e-16
filtered_inv_coulter,929.0,36.0,39.851416,6.107390,-0.712484,7.619174e-01
filtered_lab1_coulter,97.0,0.0,4.430351,2.033995,-2.423974,9.923241e-01
filtered_lab2_coulter,120.0,1.0,3.752921,1.897438,-1.714375,9.567701e-01


In [11]:
hypothesis_3(rts_colony, midProb, print_res=True)

Number of mean containing triples:  708
Mean of poission binomial:  220.313684239
SD of poission binomial:  13.4181445464
Z statistic:  36.3080241145
p-value:  0.0


In [12]:
hypothesis_3(inv_colony, midProb, print_res=True)

Number of mean containing triples:  128
Mean of poission binomial:  108.501276257
SD of poission binomial:  9.25874114803
Z statistic:  2.05197698475
p-value:  0.0200859495977


## Test for individuals in the lab

In [13]:
hypothesis_3_ind = pd.DataFrame(columns=('No. complete', 'No. mean containing', 
                                         'No.expected', "sd", "Z", "p-value"))

In [14]:
for inv in set(filtered_inv_colony.Inv):
    hypothesis_3_ind.loc[inv] = hypothesis_3(
        filtered_inv_colony.loc[filtered_inv_colony.Inv == inv].copy(), midProb)

In [15]:
hypothesis_3_ind

,No. complete,No. mean containing,No.expected,sd,Z,p-value
I,44.0,8.0,7.818411,2.480351,-0.128373,0.551073
G,8.0,4.0,1.829225,1.179072,1.417026,0.078238
A,248.0,42.0,44.087806,5.895250,-0.438965,0.669656
H,21.0,1.0,2.827193,1.554125,-1.497429,0.932859
D,77.0,13.0,13.584431,3.323176,-0.326324,0.627910
E,10.0,2.0,1.306389,1.059426,0.182751,0.427497
C,85.0,25.0,17.189069,3.656139,1.999632,0.022770
B,56.0,8.0,9.622682,2.790427,-0.760701,0.776582
F,29.0,6.0,5.139362,2.020507,0.178489,0.429169
